# REINFORCE

In this implementation, we have just one state and find optimal policy as a table of actions (shape = n_maneuvers * 4 - 1). In future, we should decide what exactly policy is - will it provides actions series by some initial state or action by objects position (in this case, what is a state).

суть - обучается сетка на состояниях и рандомных действиях
подумать, что должна выдавать сетка и куда засунуть рандом.

In [1]:
import numpy as np
import tensorflow as tf
import pykep as pk

import sys
import os
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

from api import Environment, MAX_FUEL_CONSUMPTION
from simulator import Simulator, read_space_objects
from agent import TableAgent as Agent

from copy import copy

In [2]:
start_time = 6599.95
step = 0.0001
end_time = 6600.05

osc = read_space_objects(parent_dir + "/data/collision.osc", "osc")
protected = osc[0]
debris = [osc[1]]

action_dim = 4
state_dim = (6 * (len(debris) + 1),) # positions 

start_time_mjd2000 = pk.epoch(start_time, "mjd2000")
max_fuel_cons = MAX_FUEL_CONSUMPTION
fuel_level = protected.get_fuel()

env = Environment(copy(protected), copy(debris), start_time_mjd2000)

# Building the network for REINFORCE

For REINFORCE algorithm, we'll need a model that predicts action probabilities given states. Here insteed of probabilities of actions we will generate parameters of disributions.

dV - normal

time_to_req - uniform

In [3]:
import tensorflow as tf

#create input variables. We only need <s,a,R> for REINFORCE
states = tf.placeholder('float32',(None,)+state_dim,name="states")
actions = tf.placeholder('int32',name="action_ids")
cumulative_rewards = tf.placeholder('float32', name="cumulative_returns")

In [7]:
# Network
# TODO - нормальная сеть
# TODO - что такое policy
weights = tf.Variable(
    tf.truncated_normal([state_dim[0], action_dim]))
biases = tf.Variable(tf.zeros([action_dim]))

policy = tf.matmul(states, weights) + biases
log_policy = tf.log(policy)

In [87]:
#utility function to pick action in one given state
get_action_dist_params = lambda s: policy.eval({states:[s]})[0] 

#### Loss function and updates

We now need to define objective and update over policy gradient.

Our objective function is

$$ J \approx  { 1 \over N } \sum  _{s_i,a_i} \pi_\theta (a_i | s_i) \cdot G(s_i,a_i) $$


Following the REINFORCE algorithm, we can define our objective as follows: 

$$ \hat J \approx { 1 \over N } \sum  _{s_i,a_i} log \pi_\theta (a_i | s_i) \cdot G(s_i,a_i) $$

When you compute gradient of that function over network weights $ \theta $, it will become exactly the policy gradient.


In [11]:
#REINFORCE objective function
#hint: you need to use log_policy_for_actions to get log probabilities for actions taken
# TODO - проверить
# J = списываем математику#<policy objective as in the last formula. Please use mean, not sum.>
J = tf.reduce_mean(log_policy_for_actions * cumulative_rewards, axis=)

In [13]:
#regularize with entropy
# entropy = <compute entropy. Don't forget the sign!>
# TODO - add entropy
entropy = 1 

In [14]:
#all network weights
all_weights = tf.trainable_variables() #получаем веса чтоль?<a list of all trainable weights in your network>

#weight updates. maximizing J is same as minimizing -J. Adding negative entropy.
loss = -J -0.1*entropy #градиентынй подъем поэтому минус

update = tf.train.AdamOptimizer().minimize(loss,var_list=all_weights)

### Computing cumulative rewards

In [98]:
def get_cumulative_rewards(rewards, #rewards at each step
                           gamma = 0.99 #discount for reward
                           ):
    """
    take a list of immediate rewards r(s,a) for the whole session 
    compute cumulative rewards R(s,a) (a.k.a. G(s,a) in Sutton '16)
    R_t = r_t + gamma*r_{t+1} + gamma^2*r_{t+2} + ...
    
    The simple way to compute cumulative rewards is to iterate from last to first time tick
    and compute R_t = r_t + gamma*R_{t+1} recurrently
    
    You must return an array/list of cumulative rewards with as many elements as in the initial rewards.
    """
    '''
    <your code here>
    считаем рекурсивно с конца
    
    G_t = r_t
    G_t-1 = r_t - 1
    '''
    G = np.zeros(len(rewards))
    for i in range(len(rewards))[::-1]:
        if i == len(rewards)-1:
            G[i] = rewards[i]
        else:
            G[i] = rewards[i] + gamma*G[i+1]
    return G
    

In [99]:
assert len(get_cumulative_rewards(range(100))) == 100
assert np.allclose(get_cumulative_rewards([0,0,1,0,0,1,0],gamma=0.9),[1.40049, 1.5561, 1.729, 0.81, 0.9, 1.0, 0.0])
assert np.allclose(get_cumulative_rewards([0,0,1,-2,3,-4,0],gamma=0.5), [0.0625, 0.125, 0.25, -1.5, 1.0, -4.0, 0.0])
assert np.allclose(get_cumulative_rewards([0,0,1,2,3,4,0],gamma=0), [0, 0, 1, 2, 3, 4, 0])
print("looks good!")

looks good!


In [100]:
def train_step(_states,_actions,_rewards):
    """given full session, trains agent with policy gradient"""
    _cumulative_rewards = get_cumulative_rewards(_rewards)
#     print(len(_states), _actions.shape, _cumulative_rewards.shape)
    update.run({states:_states,actions:_actions,cumulative_rewards:_cumulative_rewards})

### Playing the game

In [101]:
def get_state(epoch, protected, debris):
    state = np.array(protected.position(start_time_mjd2000)).reshape(-1)
    for d in debris:
        state = np.hstack([
            state, np.array(d.position(start_time_mjd2000)).reshape(-1)
        ])
    return state

In [102]:
def generate_session():
    """play env with REINFORCE agent and train at the session end"""
    
    #arrays to record session
    states,actions,rewards = [],[],[]
    
    env = Environment(copy(protected), copy(debris), start_time_mjd2000)
    s = get_state(start_time_mjd2000, protected, debris)
    a = np.empty(4)
    
    # TODO - end in session and/or time end (for infinite learning)
    for t in range(10):
        
        #action probabilities array aka pi(a|s)
        # TODO - true distributions and constraints
        action_dist_params = get_action_dist_params(s)
        for i in range(3):
            a[i] = np.random.normal(action_dist_params[i], 0.1)
        max_time_to_req = max(0.02, action_dist_params[3])
        a[3] = np.random.uniform(0.01, max_time_to_req, 1)
        
        # досматриваем симуляцию до time_to_req
        # TODO - сделать нормально
        new_s = s
        r = 1

        
        #record session history to train later
        states.append(s)
        actions.append(a)
        rewards.append(r)
        
        s = new_s

            
    train_step(states,actions,rewards)
            
    return sum(rewards)
        

In [103]:
s = tf.InteractiveSession()
s.run(tf.global_variables_initializer())

for i in range(3):
    
    rewards = [generate_session() for _ in range(2)] #generate new sessions
    
    # TODO - reward
    print ("mean reward:%.3f"%(np.mean(rewards)))
        


InvalidArgumentError: Shapes of all inputs must match: values[0].shape = [10] != values[1].shape = [10,4]
	 [[Node: stack_1 = Pack[N=2, T=DT_INT32, axis=-1, _device="/job:localhost/replica:0/task:0/device:GPU:0"](range_1, _arg_action_ids_0_0/_7)]]

Caused by op u'stack_1', defined at:
  File "/home/dev/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/dev/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/dev/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/dev/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/dev/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/dev/anaconda2/lib/python2.7/site-packages/zmq/eventloop/ioloop.py"

### Results & video

In [ ]:
#record sessions
import gym.wrappers
env = gym.wrappers.Monitor(gym.make("CartPole-v0"),directory="videos",force=True)
sessions = [generate_session() for _ in range(100)]
env.close()


In [ ]:
#show video
from IPython.display import HTML
import os

video_names = list(filter(lambda s:s.endswith(".mp4"),os.listdir("./videos/")))

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format("./videos/"+video_names[-1])) #this may or may not be _last_ video. Try other indices

In [ ]:
#That's all, thank you for your attention!